<a href="https://colab.research.google.com/github/r-40021/janken-ai/blob/main/janken.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

まだまだ未熟な AI のため、環境によっては認識しなかったり、誤認識したりすることがあります。

In [ ]:
"""
======================
依存関係の読み込み
======================
"""
!pip install ultralytics

from ultralytics import YOLO
from IPython.display import display, HTML, clear_output
from google.colab.output import eval_js
from base64 import b64decode
import cv2
import numpy as np
from google.colab import drive
import logging
import time
import random

"""
======================
モデルのダウンロード
======================
"""
!wget https://github.com/r-40021/janken-ai/raw/main/janken-best.pt

clear_output() # コンソールクリア
time.sleep(0.5)

"""
======================
じゃんけんを行う
======================
""" 
# 勝敗を判定する関数 (参考：https://python.atelierkobato.com/janken/)
def judge(player_hand, computer_hand):
  score = (player_hand - computer_hand + 3) % 3
  if score == 0:
    return '引き分け'
  elif score == 1:
    return '負け'
  else:
    return '勝ち'

# じゃんけんを行うメイン関数
def janken(filename='photo.jpg', quality=0.8):
  html = HTML('''
    <style>
      .visual-area {
        display: grid;
        grid-template-columns: 1fr 1fr;
        place-items: center;
      }
      .img-area>img {
        display: none;
        margin: auto;
      }
      .img-area[data-hand="0"] .goo-img{
        display: block;
      }
      .img-area[data-hand="1"] .choki-img{
        display: block;
      }
      .img-area[data-hand="2"] .par-img{
        display: block;
      }
      .hand-text {
        font-width: bold;
        font-size: 1.5rem;
      } 
      .camera {
        transform: scaleX(-1);
        -webkit-transform: scaleX(-1);
      }
    </style>
    <h1>AI じゃんけん</h1>
    <h3>現在の戦績：<span id="result-summary">0戦中 0勝 0敗 0分 (勝率:--)</span></h2>
    <p>カメラに手を映し、カメラ映像をクリックしてください。</p>
    <div class="visual-area">
      <video id="camera" class="camera">
        カメラ映像
      </video>
      <div class="img-area" id="img-area">
        <img class="goo-img" src="https://raw.githubusercontent.com/r-40021/janken-ai/main/img/goo.png" width="70%" alt="グー">
        <img class="choki-img" src="https://raw.githubusercontent.com/r-40021/janken-ai/main/img/choki.png" width="70%" alt="チョキ">
        <img class="par-img" src="https://raw.githubusercontent.com/r-40021/janken-ai/main/img/par.png" width="70%" alt="パー">
      </div>
      <p id="player-hand" class="hand-text"></p>
      <p id="computer-hand" class="hand-text">コンピューター</p>
    </div>
    <script>
      // 表示領域を調整
      google.colab.output.setIframeHeight(document.documentElement.scrollHeight, true);

      async function takePhoto(quality) {
        const video = document.getElementById('camera')
        const stream = await navigator.mediaDevices.getUserMedia({video: true});

        video.srcObject = stream;
        await video.play(); // カメラ映像を再生

        // キャプチャされるまで待つ
        await new Promise((resolve) => {
          document.getElementById('camera').addEventListener('click', () => resolve());
        });

        const canvas = document.createElement('canvas');
        canvas.width = video.videoWidth;
        canvas.height = video.videoHeight;
        canvas.getContext('2d').drawImage(video, 0, 0);
        stream.getVideoTracks()[0].stop();
        return canvas.toDataURL('image/jpeg', quality);
      }

      // 人間の手を更新する関数
      function updatePlayerHand(handId) {
        updateHand(handId, 'player-hand');
      }

      // コンピュータの手を更新する関数
      function updateComputerHand(handId) {
        updateHand(handId, 'computer-hand');
        document.getElementById('img-area').dataset.hand = handId;
      }

      // 出した手を更新する関数
      function updateHand(handId, targetId) {
        let handName = handId;
        switch (handId) {
          case 0:
            handName = 'グー';
            break;
          case 1:
            handName = 'チョキ';
            break;
          case 2:
            handName = 'パー';
            break;
          case -1:
            handName = '認識失敗';
            break; 
        }
        document.getElementById(targetId).textContent = handName;
      }

      // 戦績を更新する関数
      function updateResult(str) {
        document.getElementById('result-summary').textContent = str;
      }
    </script>
  ''')
  player_name = input('名前を入力して下さい: ') or 'ゲスト' # プレーヤー名を尋ねて player_name に代入
  display(html) # UI の描画
  eval_js('updatePlayerHand("{}")'.format(player_name)) # プレーヤー名を表示する
  logging.getLogger("ultralytics").setLevel(logging.WARNING) # 標準レベルのログを表示しない
  allResults = {
    'win': 0,
    'lose': 0,
    'draw': 0
  } # 戦績を格納する

  while True:
    data = eval_js('takePhoto({})'.format(quality)) # カメラを起動する
    binary = b64decode(data.split(',')[1])
    img_np = cv2.imdecode(np.frombuffer(binary, dtype='uint8'), cv2.IMREAD_UNCHANGED) # 推論対象の画像
    model = YOLO('janken-best.pt')
    predictResults = model.predict(source=img_np) # 推論する
    result = predictResults[0].boxes
    class_list = result.cls # クラス一覧
    if len(class_list) == 0:
      # 認識できなかった場合
      eval_js('updatePlayerHand({})'.format(-1))
      eval_js('updateComputerHand("{}")'.format('コンピューター'))
    else:
      conf_list = result.conf # 推論の信頼度の配列
      player_hand = class_list[conf_list.argmax()].numpy() # 人間の手
      computer_hand = random.randint(0, 2) # コンピュータの手
      result = judge(player_hand, computer_hand) # 勝敗判定
      
      # 戦績更新処理
      if result == '勝ち':
        allResults['win'] += 1
      elif result == '負け':
        allResults['lose'] += 1
      elif result == '引き分け':
        allResults['draw'] += 1
      
      # 表示内容の書き換え
      eval_js('updatePlayerHand({})'.format(player_hand)) # AI 推論結果を基に、人間が出した手を表示する
      eval_js('updateComputerHand({})'.format(computer_hand)) # コンピュータが出した手を表示する
      win = allResults['win'] # 勝ちの数
      lose = allResults['lose'] # 負の数
      draw = allResults['draw'] # 引き分けの数
      total = win + lose + draw # 合計試合数
      eval_js('updateResult("{}戦中 {}勝 {}敗 {}分 (勝率:{})")'.format(total, win, lose, draw, win/total)) # 戦績を更新する

janken() # メイン関数を実行する